In [94]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import pathlib
import os



In [2]:
df = pd.read_csv('HousingPrices.csv')
X = df.drop(columns=['SalePrice'])
ydf = df[['SalePrice']]
df.head()



,YearBuilt,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,YrSold,SalePrice
0,2003,854,1710,2,1,3,8,2008,208500
1,1976,0,1262,2,0,3,6,2007,181500
2,2001,866,1786,2,1,3,6,2008,223500
3,1915,756,1717,1,0,3,7,2006,140000
4,2000,1053,2198,2,1,4,9,2008,250000


In [4]:

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, ydf, test_size = 0.2)

In [5]:
xval, x_test, yval, y_test = train_test_split(xtest, ytest, test_size = 0.5)


     

In [71]:

    
    model = keras.Sequential()
    model.add(keras.layers.Dense(units= 8,activation= 'relu', input_shape = (8,)))
    model.add(keras.layers.Dense(units= 8, activation= 'relu'))
    model.add(keras.layers.Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_squared_error'])
    model.summary()
   

Model: &quot;sequential_12&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_25 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 9         
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [72]:
 model.fit(xtrain,ytrain, epochs=30,validation_data=(xval,yval))
    

Epoch 1/30
37/37 [==============================] - 0s 7ms/step - loss: 39441285120.0000 - mean_squared_error: 39441285120.0000 - val_loss: 39472578560.0000 - val_mean_squared_error: 39472578560.0000
Epoch 2/30
37/37 [==============================] - 0s 3ms/step - loss: 39351472128.0000 - mean_squared_error: 39351472128.0000 - val_loss: 39364878336.0000 - val_mean_squared_error: 39364878336.0000
Epoch 3/30
37/37 [==============================] - 0s 3ms/step - loss: 39210098688.0000 - mean_squared_error: 39210098688.0000 - val_loss: 39184990208.0000 - val_mean_squared_error: 39184990208.0000
Epoch 4/30
37/37 [==============================] - 0s 3ms/step - loss: 38963326976.0000 - mean_squared_error: 38963326976.0000 - val_loss: 38860902400.0000 - val_mean_squared_error: 38860902400.0000
Epoch 5/30
37/37 [==============================] - 0s 3ms/step - loss: 38571884544.0000 - mean_squared_error: 38571884544.0000 - val_loss: 38396391424.0000 - val_mean_squared_error: 38396391424.0000


&lt;tensorflow.python.keras.callbacks.History at 0x2015ab578c8&gt;

In [76]:
test_data = np.array([2003,	854,	1710,	2,	1,	3,	8,	2008])
print(model.predict(test_data.reshape(1,8)))

[[203592.88]]


In [83]:
keras_file = 'linear.h5'
keras.models.save_model(model,keras_file)


In [98]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()



Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\NAMANB~1\AppData\Local\Temp\tmp5he5e99p\assets


In [99]:
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

2172